# AI Lab Project
#### Ali Raza - 20I-0782
#### Haris Shahid - 20I-0663

In [1]:
%pip install Flask



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from math import sqrt
import random


### Reading Datasets

In [3]:
aircraftData = pd.read_csv("aircraftDataset.csv")
aircraftData = pd.DataFrame(aircraftData)


airportData = pd.read_csv("airportDataset.csv")
airportData = pd.DataFrame(airportData)


weatherData = pd.read_csv("weatherDataset.csv")
weatherData = pd.DataFrame(weatherData)


In [4]:
aircraftData.head(5)

#=========================Data Interpretation==============================

# Aircraft Type	                    | Max Range (In KM)                       | Cruise Speed (Kmph)     | Fuel Consumption at Cruise (litres per hr) 
# Uniquely identifies each aircraft | Max distance possible without refueling |  Speed at cruise height | Fuel consumed per hr       

# ICAO CODES
# Aircraft is available at this airport

#aircraftData['Aircraft Type'].where(aircraftData['ICAO CODES'] == 'ISLA')

,Aircraft Type,Max Range,Cruise Speed,Fuel Consumption at Cruise,ICAO CODES
0,Aircraft 1,3524.704974,440.407155,1084.211636,MULT
1,Aircraft 2,7463.252518,474.475130,494.213927,KARA
2,Aircraft 3,7810.298939,463.793756,402.758677,PESH
3,Aircraft 4,1442.066536,188.000000,788.008426,SIAL
4,Aircraft 5,6261.240012,411.286311,993.237674,SUKK


In [5]:
#=========================Data Interpretation==============================

# Date            | Wind Speed | Wind Direction     | City 
# Departure date  | In kmph    | In degrees (0-360) | relavent city      



In [6]:
airportData.head(5)

#=========================Data Interpretation==============================

# ICAO                        | Latitude (In km)                                        | Longitude                                              | City 
# Uniquely identifies airport | Latitude distance from airport to city in city column   | Longitude distance from airport to city in city column | destination city      


,ICAO Code,Latitude,Longitude,City
0,ISLA,2580,2886,Peshawar
1,ISLA,3785,2691,Karachi
2,ISLA,2225,264,Lahore
3,ISLA,106,2616,Quetta
4,ISLA,1799,2184,Multan


### Preprocessing

In [7]:
# Remove rows where 'fuel consumption', 'cruise speed', and 'max range' are NaN
aircraftData = aircraftData.dropna(subset=['Fuel Consumption at Cruise', 'Cruise Speed', 'Max Range'])

#Bring wind direction in range(0,360) degrees 
weatherData['Wind Direction'] = weatherData['Wind Direction'] % 360
weatherData.head(5)

,Date,Wind Speed,Wind Direction,City
0,2007-01-01,24.316468,255.233176,Multan
1,2007-01-01,28.748364,104.971560,Islamabad
2,2007-01-01,27.188154,170.915093,Gujranwala
3,2007-01-01,8.418233,268.304630,Karachi
4,2007-01-01,1.945127,288.329295,Sialkot


### Supporting Functions

In [8]:
def euclidean_distance(coords):
    
    lat1, lon1 = coords 
    distance = sqrt((lat1)**2+(lon1)**2)

    return distance



### Fitness Function

In [9]:
#Smaller fitness is better

def fitness(route,date,aircraftData=aircraftData,weatherData=weatherData):

    fitness = 0

    fuel_weightage = 0.7
    time_weightage = 0.3

    #Weather wind speed categories (see report for details on chosen ranges)
    max_safe = 15
    max_acceptable = 30
    max_cautionary = 45
    max_unsafe = 60
    #Else impossible/ too dangerous 

    src_weather_weight = 1
    dest_weather_weight = 1

    


    for i in range(len(route)-1):

        curr_src = route[i]
        curr_dest = route[i+1]

        print("\n\n==================================",curr_src," TO ",curr_dest,"==================================\n\n")

        #--------------------------------------------------------------Aircraft fitness---------------------------------------------------

        #Distance between current airport and next airport in the route
        distance = euclidean_distance(airportData.loc[(airportData['ICAO Code'] == curr_src) & (airportData['City'].str.upper().str[:4] == curr_dest), ['Latitude', 'Longitude']].values.flatten())
        print("Distance of journey",distance, "KM")

        #Eligible aircrafts
        aircrafts = aircraftData.loc[(aircraftData['ICAO CODES'] == curr_src) & (aircraftData['Max Range'] >= distance),'Aircraft Type'].values

        print("Eligible aircrafts:\n",aircrafts)
        print(len(aircrafts),"aircrafts available at airport ",curr_src,"\n")

        #Returing -1 means the route is unusable
        if(len(aircrafts) == 0):
            return -1

        #Best fuel efficiency
        min_consumption = float(aircraftData.loc[aircraftData['Aircraft Type'].isin(aircrafts.flatten()), 'Fuel Consumption at Cruise'].min())

        #print(min_consumption)
        best_aircraft = aircraftData.loc[aircraftData['Fuel Consumption at Cruise'] == min_consumption].reset_index(drop=True)
        print("\nBest aircraft for journey :\n", best_aircraft)

        #Time taken in hrs
        time = float(distance / (best_aircraft['Cruise Speed'].iloc[0]))
        print("\nDuration of flight:", time, "Hrs")
        print("\nFuel consumption:",min_consumption*time, "Litres")

        #Aircraft fitness (Scaling time in hrs by 100) | (min_consumption is in litres per hr)
        aircraft_fitness = ((time*100)*time_weightage) + ((min_consumption*time)*fuel_weightage)

        #--------------------------------------------------------------Weather fitness---------------------------------------------------
        #source weather
        src_wind_speed = float(weatherData.loc[(weatherData['City'].str.upper().str[:4] == curr_src) & (weatherData['Date'] == date),'Wind Speed'].iloc[0])
        print("\nWind speed in (SOURCE)", curr_src, ":",src_wind_speed)

        #destination weather
        dest_wind_speed = float(weatherData.loc[(weatherData['City'].str.upper().str[:4] == curr_dest) & (weatherData['Date'] == date),'Wind Speed'].iloc[0])
        print("\nWind speed in (DESTINATION)", curr_dest, ":",dest_wind_speed)

        #Source weather effects
        if(np.isnan(src_weather_weight)): #If no data is present then must be cautious hence the weight is same as that of cautionary category 
            src_weather_weight = 1.5
        elif(src_wind_speed <= max_safe):
            src_weather_weight = 1
        elif(src_wind_speed <= max_acceptable):
            src_weather_weight = 1.25 
        elif(src_wind_speed <= max_cautionary | np.isnan(src_weather_weight)):
            src_weather_weight = 1.5
        elif(src_wind_speed <= max_unsafe):
            src_weather_weight = 1.75
        elif(src_wind_speed > max_unsafe):
            #Impossible
            return -1 #Returing -1 means the route is unusable
        
        #Destination weather effects
        if(np.isnan(dest_weather_weight)): #If no data is present then must be cautious hence the weight is same as that of cautionary category 
            dest_weather_weight = 1.5
        elif(dest_wind_speed <= max_safe):
            dest_weather_weight = 1
        elif(dest_wind_speed <= max_acceptable):
            dest_weather_weight = 1.25 
        elif(dest_wind_speed <= max_cautionary):
            dest_weather_weight = 1.5
        elif(dest_wind_speed <= max_unsafe):
            dest_weather_weight = 1.75
        elif(dest_wind_speed > max_unsafe):
            #Impossible
            return -1 #Returing -1 means the route is unusable

        local_fitness = aircraft_fitness*src_weather_weight*dest_weather_weight
        fitness += local_fitness
        print("\n====> FITNESS =",local_fitness,"<====")

    return fitness

### Core Functions

In [10]:
def init_pop(src, dest, population_size):
    
    # Initialize 50 possible routes randomly with random lengths
    population = []

    for _ in range(population_size):
        # Create a random route length between 2 and the total number of airports
        route_length = random.randint(2, len(airportData['ICAO Code'].unique()))
        
        # Create a random route with the specified length, excluding src and dest
        route = random.sample([city for city in airportData['ICAO Code'].unique() if city not in [src, dest]], route_length - 2)
        route = [src] + route + [dest]
        population.append(route)

    return population

def selection(population, fitness_values):

    # Select the best 2 routes based on fitness (lower fitness is better)
    sorted_indices = np.argsort(fitness_values)
    selected_routes = [population[i] for i in sorted_indices[:2]]

    return selected_routes


def crossover(parent1, parent2, src, dest):

    # Randomly select a crossover point between 1 and the minimum length of the parents
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    
    # create children
    child1 = parent1[1:crossover_point] + [city for city in parent2 if city not in parent1[1:crossover_point]]
    child2 = parent2[1:crossover_point] + [city for city in parent1 if city not in parent2[1:crossover_point]]
    
    # Remove all instances of src and dest from the children except for the first and last indices
    child1 = [src] + [city for city in child1[1:-1] if city != src] + [dest]
    child1 = [city for city in child1[0:-1] if city != dest] + [dest]

    child2 = [src] + [city for city in child1[1:-1] if city != src] + [dest]
    child2 = [city for city in child1[0:-1] if city != dest] + [dest]
    return child1, child2



def mutation(route):

    # Randomly decide whether to:
    # 1. Remove a connection
    # 2. Add a new connection (least likely)
    # 3. Change a random existing connection

    mutation_type = random.choice([1,2,3])

    if mutation_type == 1 and len(route) > 3:
        # Remove a connection
        remove_index = random.randint(1, len(route) - 2)
        route.pop(remove_index)
    elif mutation_type == 2:
        # Add a new connection
        new_city = random.choice([city for city in airportData['ICAO Code'].unique() if city not in route])
        insert_index = random.randint(1, len(route) - 1)
        route.insert(insert_index, new_city)
    elif mutation_type == 3 and len(route) > 2:
        # Change a random existing connection
        change_index = random.randint(1, len(route) - 2)
        new_city = random.choice([city for city in airportData['ICAO Code'].unique() if city != route[change_index]])
        route[change_index] = new_city

    return route


def replace(population, children, fitness_values):

    # Replace worst 2 from the population with the new children
    sorted_indices = np.argsort(fitness_values)
    population[sorted_indices[-2]] = children[0]
    population[sorted_indices[-1]] = children[1]

    return population


### Driver Function

In [11]:
import random

def geneticAlgo(src, dest, date):

    population_size = 20
    generations = 1

    # Initialize population
    population = init_pop(src,dest,population_size)
    print(population)
        
    for generation in range(generations):
        
        # Evaluate fitness of each route in the population
        fitness_values = [fitness(route, date) for route in population]
        #print(fitness_values)

        # Select the best routes
        selected_routes = selection(population, fitness_values)
        #print(selected_routes)

        # Perform crossover to create children
        children = []
        for i in range(0, len(selected_routes), 2):
            child1, child2 = crossover(selected_routes[i], selected_routes[i + 1], src, dest)
            children.extend([child1, child2])
        
        print(children)

        # Perform mutation on children
        mutated_children = [mutation(child) for child in children]

        print(mutated_children)

        # Replace the worst routes in the population with the new children
        population = replace(population, mutated_children, fitness_values)

        # Print the best fitness in each generation
        best_fitness = min(fitness_values)
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")

    # Select the best route after all generations
    best_route_index = np.argmin(fitness_values)
    optimal_route = population[best_route_index]

    # Extract the next best 5 routes
    sorted_routes_indices = np.argsort(fitness_values)
    alt_routes_indices = sorted_routes_indices[1:6]
    alt_routes = [population[i] for i in alt_routes_indices]

    return optimal_route, alt_routes



### Visualization and Frontend

In [13]:
from flask import Flask, render_template, request

app = Flask(__name__)


@app.route('/', methods=['GET', 'POST'])
def optimizeRoute():
    if request.method == 'POST':
        try:
            #Extract user input from webpage
            source = request.form['source']
            destination = request.form['destination']
            date = request.form['date']
            
            #Call genetic algorithm function
            result, alt_routes = geneticAlgo(source,destination,date)
            alt1 = alt_routes[0]
            alt2 = alt_routes[1]
            alt3 = alt_routes[2]
            alt4 = alt_routes[3]
            alt5 = alt_routes[4]

            #Render page to display optimal and alternative routes
            return render_template('front.html', result=result, alt1=alt1, alt2=alt2, alt3=alt3, alt4=alt4, alt5=alt5)
        except ValueError:
            error_message = "Invalid input. Please enter valid values for source, destination, and date."
            return render_template('front.html', error_message=error_message)
    else:
        return render_template('front.html')


if __name__ == '__main__':
    try:
        app.run(debug=True, use_reloader=False)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [06/Dec/2023 14:28:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2023 14:28:15] "GET /static/peakpx.jpg HTTP/1.1" 304 -
127.0.0.1 - - [06/Dec/2023 14:28:15] "GET /favicon.ico HTTP/1.1" 404 -


[['ISLA', 'MULT', 'KARA'], ['ISLA', 'GUJR', 'GWAD', 'MULT', 'FAIS', 'KARA'], ['ISLA', 'RAWA', 'GWAD', 'QUET', 'SUKK', 'MULT', 'PESH', 'GUJR', 'KARA'], ['ISLA', 'QUET', 'MULT', 'FAIS', 'LAHO', 'PESH', 'KARA'], ['ISLA', 'GUJR', 'GWAD', 'MULT', 'PESH', 'LAHO', 'RAWA', 'SUKK', 'QUET', 'FAIS', 'SIAL', 'KARA'], ['ISLA', 'RAWA', 'SIAL', 'QUET', 'SUKK', 'MULT', 'GWAD', 'PESH', 'LAHO', 'KARA'], ['ISLA', 'PESH', 'SUKK', 'MULT', 'KARA'], ['ISLA', 'MULT', 'RAWA', 'FAIS', 'QUET', 'LAHO', 'GUJR', 'SUKK', 'PESH', 'KARA'], ['ISLA', 'SUKK', 'RAWA', 'SIAL', 'LAHO', 'QUET', 'PESH', 'GWAD', 'MULT', 'FAIS', 'KARA'], ['ISLA', 'FAIS', 'MULT', 'RAWA', 'GWAD', 'GUJR', 'LAHO', 'KARA'], ['ISLA', 'FAIS', 'QUET', 'SUKK', 'GUJR', 'KARA'], ['ISLA', 'SIAL', 'MULT', 'SUKK', 'FAIS', 'PESH', 'LAHO', 'KARA'], ['ISLA', 'MULT', 'KARA'], ['ISLA', 'GUJR', 'FAIS', 'MULT', 'SUKK', 'SIAL', 'RAWA', 'LAHO', 'KARA'], ['ISLA', 'SIAL', 'MULT', 'LAHO', 'SUKK', 'GUJR', 'GWAD', 'PESH', 'FAIS', 'KARA'], ['ISLA', 'LAHO', 'MULT', 'RAWA', 

127.0.0.1 - - [06/Dec/2023 14:30:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2023 14:30:41] "


Wind speed in (SOURCE) RAWA : 5.084001221175267

Wind speed in (DESTINATION) KARA : 8.418232851835846

====> FITNESS = 2853.80036897127 <====
[['ISLA', 'MULT', 'KARA'], ['ISLA', 'MULT', 'KARA']]
[['ISLA', 'KARA', 'KARA'], ['ISLA', 'MULT', 'KARA']]
Generation 1, Best Fitness: 2639.9327860376625


GET /static/peakpx.jpg HTTP/1.1" 304 -
